In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
import zipfile
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set()

sns.set_context("talk")
import re

# Ensure that Pandas shows at least 100 characters in columns
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

from pathlib import Path

# SQL
import sqlalchemy

In [20]:
alameda = pd.read_csv('alameda_v2.csv', sep='|')

/usr/local/linux/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,14,17,19,22,40,43,48,51,62,70,73,74,79,102,123) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
#Filter for Berkeley JCE only
berk_jc = alameda[alameda.PropertyCity == 'BERKELEY']
print('Current number:', len(berk_jc))

Current number: 26994


In [74]:
# Filter for not subsidized
berk_jc = berk_jc[berk_jc['Subsidized'] == 'N']

In [75]:
# Check structure 
# (number of records, number of columns)
berk_jc.shape

(26993, 135)

In [76]:
data_dict = pd.read_excel('ZAsmt_DataDictionary_2016-01.xlsx', sheet_name='LandUse', header=5, usecols=np.arange(4))
data_dict.head(2)

,StndCode,PropertyLandUse Description,Prefix Code Classification,Classification
0,AG000,AGRICULTURAL GENERAL,AG,Agricultural
1,AG101,FARM (IRRIGATED OR DRY),AG,Agricultural


In [77]:
residential = data_dict[data_dict['Prefix Code Classification'].isin(['RI', 'RR'])]
R = residential['StndCode']

In [78]:
# Quoted comments refer to the lines in the JC and RC inventory. Specifically each coverage/exemption.

# "Units owned by a government agency"
# "Nursing or hospital units or home for the aged and the like"
# Filter rows to only include residential buildings
berk_jc = berk_jc[berk_jc['PropertyLandUseStndCode'].isin(R)]
print('Current number:', len(berk_jc))

Current number: 26383


In [79]:
# "Two unit properties where one unit is owner-occupied"
# Filter out properties with 2 or more units and owner occupied
berk_jc = berk_jc[~(berk_jc['OccupancyStatusStndCode'].isin(['O', 'I']) & (berk_jc.NoOfUnits >= 2))]
print('Current number:', len(berk_jc))

Current number: 23046


In [65]:
berk_rc = alameda[alameda.PropertyCity == 'BERKELEY']
print('Current number:', len(berk_rc))

Current number: 26994


In [66]:
# Filter for not subsidized
berk_rc = berk_rc[berk_rc['Subsidized'] == 'N']
print('Current number:', len(berk_rc))

Current number: 26993


In [67]:
# Quoted comments refer to the lines in the JC and RC inventory. Specifically each coverage/exemption.

# Filter rows to only include residential buildings
berk_rc = berk_rc[berk_rc['PropertyLandUseStndCode'].isin(R)]
print('Current number:', len(berk_rc))

Current number: 26383


In [68]:
# "Units eligible for RC:"
# "Most multi-unit properties that were built before June 1980"
# "Units eligible for RC: Single family homes with tenants who moved in prior to 1996"

# "Units NOT eligible for RC: "
# "Single-family homes first re-rented on or after 1/1/1996"
# APPROXIMATING THOSE ABOVE^^^^^^^^
# Select buildings before 1996, this automatically include all buildings built before 1980. 
# It will also approximate buildings where people rented or moved in before 1996
berk_rc = berk_rc[berk_rc.YearBuilt < 1996]
print('Current number:', len(berk_rc))

Current number: 23900


In [72]:
berk_rc = berk_rc[(berk_rc.PropertyLandUseStndCode.str.contains('RI') & (berk_rc.YearBuilt < 1981)) | (berk_rc.PropertyLandUseStndCode.isin(['RR101', 'RR999']) & berk_rc.OccupancyStatusStndCode.isnull())]
print('Current number:', len(berk_rc))

Current number: 6247


In [80]:
# Costa Hawkins exclude Condos
berk_rc = berk_rc[berk_rc.PropertyLandUseStndCode != 'RR106']
print('Current number:', len(berk_rc))

Current number: 6247


In [81]:
berk_rc[berk_rc.PropertyLandUseStndCode != 'RR107'].shape

(6247, 135)

In [ ]:
berk_rc.OccupancyStatusStndCode.isnull().sum()